# **Requirements**

Python 3.8.10

pythonnet 2.5.2

`pip install pythonnet==2.5.2`

In [35]:
from base_interactive import PythonZOSConnection
from beam_shaper import BeamShaper
import sys
from importlib import metadata

print("Python version: ", sys.version)
print('Pythonnet version:', metadata.version('pythonnet'))

Python version:  3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
Pythonnet version: 2.5.2


# **Connect to Zemax and initiate a system**

In [36]:
zos = PythonZOSConnection()
ZOSAPI = zos.ZOSAPI
TheApplication = zos.TheApplication
TheSystem = zos.TheSystem

# TheSystem.Mode = ZOSAPI.ZOSAPI_Mode.Plugin
TheSystem.UpdateMode = ZOSAPI.LensUpdateMode.AllWindows

print('Connected to OpticStudio')
# The connection should now be ready to use.  For example:
print('Serial #: ', TheApplication.SerialCode)

print('ZOSAPI Mode: ', TheSystem.Mode)
print("Lens update mode: ", TheSystem.UpdateMode)

Connected to OpticStudio
Serial #:  20120530
ZOSAPI Mode:  0
Lens update mode:  2


# **Set System Data**

In [37]:
# Define System Explore
SysExplore = TheSystem.SystemData

## **Set Title and Notes**

In [38]:
# Set Title and Notes
SysExplore.TitleNotes.Title = "Galilean Beam Shaper"
SysExplore.TitleNotes.Notes = "Galilean beam shaper design, turning a Gaussian into a flat-top beam."
SysExplore.TitleNotes.Author = "Ziyi Xiong"

## **Set Aperture**

In [39]:
# Set Aperture
SysExplore.Aperture.ApertureType = ZOSAPI.SystemData.ZemaxApertureType.EntrancePupilDiameter
SysExplore.Aperture.ApertureValue = 4.05

SysExplore.Aperture.ApodizationType = 1 # 0: Uniform, 1: Gaussian, 2: Cosine Cubed
SysExplore.Aperture.ApodizationFactor = 1 # Apodization factor for Gaussian

## **Set Fields**

In [40]:
# Set Fields
SysExplore.Fields.SetFieldType(ZOSAPI.SystemData.FieldType.ParaxialImageHeight)
#SysExplore.Fields.ApplyFieldWizard(ZOSAPI.SystemData.FieldPattern.EqualAreaY, 9, 6.6, 0, 0, 0, True, False)

## **Set Wavelengths**

### **Via Presets**

In [41]:
# SysExplore.Wavelengths.SelectWavelengthPreset(ZOSAPI.SystemData.WavelengthPreset.FdC_Visible)

### **Customize Wavelengths**

In [42]:
SysExplore.Wavelengths.GetWavelength(1).Wavelength = 0.532

### **Remove Wavelengths**

In [43]:
# if num_wavelengths > 1: [SysExplore.Wavelengths.RemoveWavelength(i) for i in range(num_wavelengths, 1, -1)]

# **Set Lens Data**

In [44]:
SysLDE = TheSystem.LDE

## **Add Surfaces**

In [45]:
num_surfaces = SysLDE.NumberOfSurfaces

if num_surfaces == 3:
    for i in range(4): # range(4) = [0, 1, 2, 3]
        SysLDE.AddSurface()

num_surfaces = SysLDE.NumberOfSurfaces
print("Number of surfaces after insertion: ", num_surfaces)

Number of surfaces after insertion:  7


## **Get and Set Surfaces**

In [46]:
# Use a list to store all surfaces objects
Surface=[SysLDE.GetSurfaceAt(i) for i in range(0, num_surfaces)] 

## **Set Surface Data**

### **Set Stop**

In [47]:
Surface[2].IsStop = True 

### **Set Lens Data**

In [48]:
galilean = BeamShaper(omega_0 = 2.366,
                  R_max=4.05,
                  r_max=4.05,
                  d=150,
                  n=1.46071,
                  type='Galilean'
                )

Surface[1].Thickness = 10
Surface[1].Comment = "Dummy"

Surface[2].Thickness = 3
MaterialModel1 = Surface[2].MaterialCell.CreateSolveType(ZOSAPI.Editors.SolveType.MaterialModel)._S_MaterialModel
MaterialModel1.IndexNd = galilean.n
Surface[2].MaterialCell.SetSolveData(MaterialModel1)

Surface[3].Thickness = galilean.d
Surface[3].Radius = galilean.r_c1
Surface[3].Conic = galilean.k_1

Surface[4].Thickness = 3
MaterialModel2 = Surface[4].MaterialCell.CreateSolveType(ZOSAPI.Editors.SolveType.MaterialModel)._S_MaterialModel
MaterialModel2.IndexNd = galilean.n
Surface[4].MaterialCell.SetSolveData(MaterialModel2)
Surface[4].Radius = galilean.r_c2
Surface[4].Conic = galilean.k_2

Surface[5].Thickness = 10